In [1]:
# Project # 1 Jupyter Notebook
# Created by Melinda Eudy, Cameron Farquhar, Markita Francis, Luis Rojas, Kelly Rose.
# Project: 2017 Crime Data Used from several resources, UCR Crime Rates by County, US Census, and BJ Crime

In [2]:
# Dependencies and Setup *******Review before final presentation and remove what is not needed ************
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from datetime import date
from functools import reduce
import json
import ctypes
import seaborn as sns
import datetime
import gmaps
from config import g_key
import seaborn as sns
import datetime

In [3]:
# Input File (CSV)
input_data_file = "output_data/agency_crimes_stat.csv"

# Export file as a CSV
agency_crimes_stat = pd.read_csv(input_data_file)
agency_crimes_stat.dropna(inplace=True)
agency_crimes_stat.reset_index(drop=True)

,year,crime_type,agency,agency_name,county,state,state_name,latitude,longitude,age 0-9,...,age 20-29,age 30-39,age 40-49,age 50-59,age 60-69,age 70-79,age 80-89,age 90-Older,age Unknown,total_crimes
0,2017,property-crime,DE0030500,Milford Police Department,KENT; SUSSEX,DE,Delaware,38.916120,-75.421610,2,...,151,89,67,47,6,2,0,0,113,550
1,2017,violent-crime,DE0030500,Milford Police Department,KENT; SUSSEX,DE,Delaware,38.916120,-75.421610,2,...,30,14,4,3,0,0,0,0,20,105
2,2017,property-crime,DE0030300,Laurel Police Department,SUSSEX,DE,Delaware,38.677511,-75.335495,1,...,58,24,9,5,1,3,1,0,42,171
3,2017,violent-crime,DE0030300,Laurel Police Department,SUSSEX,DE,Delaware,38.677511,-75.335495,0,...,22,9,4,1,0,0,0,0,6,66
4,2017,property-crime,DE0030100,Bridgeville Police Department,SUSSEX,DE,Delaware,38.744175,-75.599144,0,...,5,5,3,1,0,0,0,0,18,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11647,2017,violent-crime,TN0290000,Grainger County Sheriff's Office,GRAINGER,TN,Tennessee,36.277463,-83.509493,0,...,5,11,8,4,2,1,0,0,0,38
11648,2017,property-crime,TN0200000,Decatur County Sheriff's Office,DECATUR,TN,Tennessee,35.603422,-88.107384,0,...,8,16,7,1,2,0,0,0,18,52
11649,2017,violent-crime,TN0200000,Decatur County Sheriff's Office,DECATUR,TN,Tennessee,35.603422,-88.107384,0,...,4,5,2,0,1,0,0,0,1,15
11650,2017,property-crime,TN0070100,Jellico Police Department,CAMPBELL,TN,Tennessee,36.586983,-84.130090,0,...,6,30,11,1,0,0,0,0,17,70


In [4]:
property_df = agency_crimes_stat.loc[agency_crimes_stat['crime_type']=='property-crime']
property_df.reset_index(drop=True)

,year,crime_type,agency,agency_name,county,state,state_name,latitude,longitude,age 0-9,...,age 20-29,age 30-39,age 40-49,age 50-59,age 60-69,age 70-79,age 80-89,age 90-Older,age Unknown,total_crimes
0,2017,property-crime,DE0030500,Milford Police Department,KENT; SUSSEX,DE,Delaware,38.916120,-75.421610,2,...,151,89,67,47,6,2,0,0,113,550
1,2017,property-crime,DE0030300,Laurel Police Department,SUSSEX,DE,Delaware,38.677511,-75.335495,1,...,58,24,9,5,1,3,1,0,42,171
2,2017,property-crime,DE0030100,Bridgeville Police Department,SUSSEX,DE,Delaware,38.744175,-75.599144,0,...,5,5,3,1,0,0,0,0,18,32
3,2017,property-crime,DE0032400,Dewey Beach Police Department,SUSSEX,DE,Delaware,38.709568,-75.078224,0,...,2,2,0,1,0,0,0,0,7,15
4,2017,property-crime,DE0030900,Blades Police Department,SUSSEX,DE,Delaware,38.633743,-75.609800,0,...,3,5,0,4,1,0,0,0,15,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6165,2017,property-crime,TN0630000,Montgomery County Sheriff's Office,MONTGOMERY,TN,Tennessee,36.526070,-87.359540,1,...,113,165,74,24,7,1,3,0,23,509
6166,2017,property-crime,TN0791000,Memphis-Shelby County Airport Authority,SHELBY,TN,Tennessee,35.183794,-89.895397,0,...,11,4,1,0,0,0,0,0,127,147
6167,2017,property-crime,TN0290000,Grainger County Sheriff's Office,GRAINGER,TN,Tennessee,36.277463,-83.509493,0,...,54,63,20,10,1,0,0,0,14,170
6168,2017,property-crime,TN0200000,Decatur County Sheriff's Office,DECATUR,TN,Tennessee,35.603422,-88.107384,0,...,8,16,7,1,2,0,0,0,18,52


In [5]:
violent_df = agency_crimes_stat.loc[agency_crimes_stat['crime_type']=='violent-crime']
violent_df.reset_index(drop=True)

,year,crime_type,agency,agency_name,county,state,state_name,latitude,longitude,age 0-9,...,age 20-29,age 30-39,age 40-49,age 50-59,age 60-69,age 70-79,age 80-89,age 90-Older,age Unknown,total_crimes
0,2017,violent-crime,DE0030500,Milford Police Department,KENT; SUSSEX,DE,Delaware,38.916120,-75.421610,2,...,30,14,4,3,0,0,0,0,20,105
1,2017,violent-crime,DE0030300,Laurel Police Department,SUSSEX,DE,Delaware,38.677511,-75.335495,0,...,22,9,4,1,0,0,0,0,6,66
2,2017,violent-crime,DE0030100,Bridgeville Police Department,SUSSEX,DE,Delaware,38.744175,-75.599144,0,...,0,1,1,0,0,0,0,0,1,8
3,2017,violent-crime,DE0032400,Dewey Beach Police Department,SUSSEX,DE,Delaware,38.709568,-75.078224,0,...,7,1,2,0,0,0,0,0,3,13
4,2017,violent-crime,DE0030900,Blades Police Department,SUSSEX,DE,Delaware,38.633743,-75.609800,0,...,4,1,0,3,0,0,0,0,2,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477,2017,violent-crime,TN0410100,Centerville Police Department,HICKMAN,TN,Tennessee,35.802291,-87.467101,0,...,5,2,2,0,1,1,0,0,0,13
5478,2017,violent-crime,TN0630000,Montgomery County Sheriff's Office,MONTGOMERY,TN,Tennessee,36.526070,-87.359540,0,...,24,23,10,8,2,1,0,0,0,76
5479,2017,violent-crime,TN0290000,Grainger County Sheriff's Office,GRAINGER,TN,Tennessee,36.277463,-83.509493,0,...,5,11,8,4,2,1,0,0,0,38
5480,2017,violent-crime,TN0200000,Decatur County Sheriff's Office,DECATUR,TN,Tennessee,35.603422,-88.107384,0,...,4,5,2,0,1,0,0,0,1,15


In [6]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = property_df[["latitude", "longitude"]]
# print(locations)

# Convert to float
total_crimes_weights = property_df["total_crimes"].astype(float)
# total_crimes_weights

# Plot Heatmap
fig = gmaps.figure(center=(40, -100), zoom_level=3.5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=total_crimes_weights, 
                                 dissipating=True, max_intensity=1,
                                 point_radius=0.5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = violent_df[["latitude", "longitude"]]
# print(locations)

# Convert to float
total_crimes_weights = violent_df["total_crimes"].astype(float)
# total_crimes_weights

# Plot Heatmap
fig = gmaps.figure(center=(40, -100), zoom_level=3.5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=total_crimes_weights, 
                                 dissipating=True, max_intensity=1,
                                 point_radius=0.5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))